In [ ]:
import joblib
import torch 
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from scipy.sparse import coo_matrix

In [ ]:
class dataSet():
    def __init__(self,feature,dim,idex,train = True):
        self.fea_name = feature
        tmp = []
        for i in idex:
            tmp.append(int(i))
        self.dim = dim
        self.train =train
        self.idex=tmp
        self.trainpath = './fea_sel/{}/{}_train-{}.pkl'.format(self.fea_name,self.fea_name.lower(),self.dim)
        self.testpath = './fea_sel/{}/{}_test-{}.pkl'.format(self.fea_name,self.fea_name.lower(),self.dim)
        labpath = './fea_sel/label/label8.pkl'
        self.label = joblib.load(labpath).iloc[self.idex]
        self.IDbank = pd.read_csv('/homes/yalin/Kaggle/trainData/emotion.csv')['tweet_id'][self.idex]
    def __len__(self):
        if self.train:
            self.bank = joblib.load(self.trainpath).toarray()
            self.bank = self.bank[self.idex]
            length = self.bank.shape[0]
        else:
            self.bank = joblib.load(self.testpath).toarray()
            length = self.bank.shape[0]
        return length
    def __getitem__(self,idx):
        coo = coo_matrix(self.bank[idx,:])
        values = coo.data
        indices = np.vstack((coo.row, coo.col))
        
        i = torch.LongTensor(indices)
        v = torch.LongTensor(values)
        shape = coo.shape

        fea = torch.sparse.LongTensor(i, v, torch.Size(shape)).to_dense()
        
        label = torch.tensor( int(self.label.iloc[idx]))
        ID = self.IDbank.iloc[idx]
        return fea,label,ID

In [ ]:
class data_prefetcher():
    def __init__(self, loader):
        self.loader = iter(loader)
        # self.stream = torch.cuda.Stream()
        self.preload()

    def preload(self):
        try:
            self.next_data = next(self.loader)
        except StopIteration:
            self.next_input = None
#            print('preload stop iteration')
            return
        # with torch.cuda.stream(self.stream):
        #     self.next_data = self.next_data.cuda(non_blocking=True)
            
    def next(self):
        # torch.cuda.current_stream().wait_stream(self.stream)
        data = self.next_data
        self.preload()
        return data   